In [84]:
# Předpoklady:
    # Uživatel nakreslí jednoduchý mnohoúhelník
    # Uživatel ho nakreslí s pořadím bodů orientovaným proti směru hodinových ručiček

In [85]:
import tkinter as tk
from tkinter import messagebox
from shapely.geometry import LineString
import numpy as np


In [104]:
# triangulation

def intersects_with_polygon(polygon, point1, point2):
    for i in range(len(polygon)):
        edge_start = polygon[i]
        edge_end = polygon[(i + 1) % len(polygon)]  # polygon is closed
        if edge_start == point1 or edge_start == point2 or edge_end == point1 or edge_end == point2: # avoid points themselves
            continue    
        if check_intersection(point1, point2, edge_start, edge_end):
            return True
    return False

def check_intersection(first1, first2, second1, second2):
    first_line = LineString([first1, first2])
    second_line = LineString([second1, second2])
    if first_line.intersects(second_line):
        return True
    return False

def is_side_correct(start, end, checked, clockwise): # check using vectors and cross product
    v1 = (end[0] - start[0], end[1] - start[1]) 
    v2 = (checked[0] - start[0], checked[1] - start[1])
    cross_product = v1[0] * v2[1] - v1[1] * v2[0]
    if (cross_product < 0 and clockwise) or (cross_product > 0 and not clockwise):
        return True
    return False

def is_clockwise(polygon):
    left = 0
    right = 0
    for i in range(len(polygon)):
        if is_side_correct(polygon[i%len(polygon)], polygon[(i+2)%len(polygon)], polygon[(i+1)%len(polygon)], True):
            left += 1
        else:
            right += 1
    if left > right:
        return True
    return False        

def triangulate_polygon(polygon):
    triangles = []
    going_clockwise = False
    if is_clockwise(polygon):
        going_clockwise = True
    i = 0
    while len(polygon) > 2: # until we can't triangulate
        vi = polygon[i % len(polygon)]
        vii = polygon[(i+1) % len(polygon)]  # polygon is closed
        viii = polygon[(i+2) % len(polygon)]
        if is_side_correct(vi, viii, vii, going_clockwise):
            if not intersects_with_polygon(polygon, vi, viii):
                triangles.append((vi, vii, viii))
                polygon.remove(vii)
                i = 0
                continue
        if going_clockwise:
            i += 1
        else:
            i -= 1
    return triangles

def sum_triangles(triangles):
    total_area = 0
    for triangle in triangles:
        area = triangle_area(triangle)
        total_area += area
    return total_area

def triangle_area(triangle):
    x1, y1 = triangle[0]
    x2, y2 = triangle[1]
    x3, y3 = triangle[2]
    area = 0.5 * abs((x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))) # shoelace formula
    return area

In [105]:
# drawing

def on_canvas_click(click):
    x = click.x
    y = click.y
    points.append((x, y))
    canvas.create_oval(x-2, y-2, x+2, y+2, fill="black")  # show the clicked point on the canvas
    


In [109]:
# buttons

def calculate_area(points, canvas, calculate_button, wrap_button, output_text):
    if len(points) > 1: # draw the polygon
        for i in range(len(points)-1):
            canvas.create_line(points[i], points[i+1], fill="black")
        canvas.create_line(points[0], points[-1:], fill="black")
        
    canvas.unbind("<Button-1>") # stops drawing
    calculate_button.config(state=tk.DISABLED)
    wrap_button.config(state=tk.DISABLED)
    triangles = triangulate_polygon(points)
    area = sum_triangles(triangles)
    text = "Area of this polygon is " + str(area) + " square pixels."
    output_text.insert(tk.END, text)
    return area

def find_wrapping(points, calculate_button, canvas):
    get_wrapping(points, canvas)
    calculate_button.config(state=tk.DISABLED)

def close_application(window):
    window.destroy()
    
def clear_canvas(canvas, points, calculate_button, wrap_button, output_text):
    canvas.delete("all")
    points.clear()
    canvas.bind("<Button-1>", on_canvas_click)
    calculate_button.config(state=tk.NORMAL)
    wrap_button.config(state=tk.NORMAL)
    output_text.delete("1.0", tk.END)  # clear text

In [110]:
# finding smallest convex wrapping

def get_wrapping(points, canvas):
    upper_wrapping = []
    lower_wrapping = []
    sorted_points = sorted(points, key=lambda p: p[0])
    upper_wrapping.append(sorted_points[0])
    upper_wrapping.append(sorted_points[1])
    lower_wrapping.append(sorted_points[0])
    lower_wrapping.append(sorted_points[1])
    for i in range(len(sorted_points)-2):
        upper_wrapping.append(sorted_points[i+2])
        lower_wrapping.append(sorted_points[i+2])
        while len(upper_wrapping)>2 and not is_side_correct(upper_wrapping[-3], upper_wrapping[-1], upper_wrapping[-2], True):
            upper_wrapping.pop(-2)
        while len(lower_wrapping)>2 and not is_side_correct(lower_wrapping[-3], lower_wrapping[-1], lower_wrapping[-2], False):
            lower_wrapping.pop(-2)
    for i in range(len(lower_wrapping)-1, 0, -1):
        upper_wrapping.append(lower_wrapping[i])
    draw_wrapping(upper_wrapping, canvas)
            
def draw_wrapping(wrapping, canvas):
    if len(wrapping) > 1: # draw the polygon
        for i in range(len(wrapping)-1):
            canvas.create_line(wrapping[i], wrapping[i+1], fill="black")
        canvas.create_line(wrapping[0], wrapping[-1], fill="black")


In [111]:
if __name__ == "__main__":
    points = []
    window = tk.Tk()
    window.title("Computational Geometry")

    canvas_width = 400
    canvas_height = 300

    canvas = tk.Canvas(window, width=canvas_width, height=canvas_height, bg="white")
    canvas.grid(row=0, column=0, columnspan=3, padx=10, pady=10)
    canvas.bind("<Button-1>", on_canvas_click)
    #canvas.bind("<Button-1>", lambda event: on_canvas_click(event, points, canvas))

    button_frame = tk.Frame(window)
    button_frame.grid(row=1, column=0, columnspan=3, padx=5, pady=10)

    calculate_button = tk.Button(button_frame, text="Calculate area", command=lambda: calculate_area(points, canvas, calculate_button, wrap_button, output_text))
    calculate_button.pack(side=tk.LEFT)

    wrap_button = tk.Button(button_frame, text="Find wrapping", command=lambda: find_wrapping(points, calculate_button, canvas))
    wrap_button.pack(side=tk.LEFT)

    clear_button = tk.Button(button_frame, text="Clear", command=lambda: clear_canvas(canvas, points, calculate_button, wrap_button, output_text))
    clear_button.pack(side=tk.LEFT)

    close_button = tk.Button(button_frame, text="Close", command=lambda: close_application(window))
    close_button.pack(side=tk.LEFT)

    output_text = tk.Text(window, height=5, width=30)
    output_text.grid(row=2, column=0, columnspan=3, padx=10, pady=10, sticky="ew")
    window.grid_columnconfigure(0, weight=1)

    window.mainloop()

upper: [(91, 171), (119, 50), (125, 234)]
lower:  [(91, 171), (119, 50), (125, 234)]
popping  (119, 50)
upper: [(91, 171), (119, 50), (125, 234), (201, 88)]
lower:  [(91, 171), (125, 234), (201, 88)]
popping  (125, 234)
upper: [(91, 171), (119, 50), (201, 88), (211, 235)]
lower:  [(91, 171), (125, 234), (201, 88), (211, 235)]
popping  (201, 88)
upper: [(91, 171), (119, 50), (201, 88), (211, 235), (271, 220)]
lower:  [(91, 171), (125, 234), (211, 235), (271, 220)]
popping  (211, 235)
upper: [(91, 171), (119, 50), (201, 88), (271, 220), (284, 182)]
lower:  [(91, 171), (125, 234), (211, 235), (271, 220), (284, 182)]
popping  (271, 220)
upper: [(91, 171), (119, 50), (201, 88), (284, 182), (328, 103)]
lower:  [(91, 171), (125, 234), (211, 235), (271, 220), (284, 182), (328, 103)]
popping  (284, 182)
popping  (201, 88)
popping  (284, 182)
upper: [(91, 171), (119, 50), (125, 234)]
lower:  [(91, 171), (119, 50), (125, 234)]
popping  (119, 50)
upper: [(91, 171), (119, 50), (125, 234), (201, 88)

In [63]:
#TODO:
    # 

In [95]:
-4 % 3

2